<a href="https://colab.research.google.com/github/Richard-Wishart/Hello-World/blob/main/jumping_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pygame
import random

# --- Constants and Initialization ---

# Initialize all imported pygame modules
pygame.init()

# Screen dimensions
SCREEN_WIDTH = 800
SCREEN_HEIGHT = 600

# Colors (using RGB values)
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
YELLOW = (255, 220, 0) # A nice chick yellow
ORANGE = (255, 165, 0)
SKY_BLUE = (135, 206, 235)
GRASS_GREEN = (50, 205, 50)

# Game window setup
screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
pygame.display.set_caption("Jumping Chick")

# Game clock for controlling the frame rate (FPS)
clock = pygame.time.Clock()
FPS = 60

# --- Player (Chick) Settings ---
chick_width = 50
chick_height = 40
# The chick stays at a fixed horizontal position
chick_x = 100
# Start the chick on the ground
ground_level = SCREEN_HEIGHT - 100
chick_y = ground_level - chick_height

# Physics variables for jumping
chick_y_velocity = 0
gravity = 0.8
jump_strength = -18 # A negative value makes the chick move up
is_jumping = False

# --- Obstacle (Egg) Settings ---
egg_width = 30
egg_height = 45
egg_speed = 7
# This list will hold all the active egg obstacles
eggs = []

# --- Font and Text Setup ---
score_font = pygame.font.Font(None, 50)
game_over_font = pygame.font.Font(None, 80)

# --- Game State Variables ---
score = 0
game_over = False
running = True

# --- Drawing Functions ---

def draw_chick(x, y):
    """Draws the chick on the screen."""
    # Chick's body
    pygame.draw.ellipse(screen, YELLOW, (x, y, chick_width, chick_height))
    # A small wing
    pygame.draw.ellipse(screen, YELLOW, (x - 10, y + 15, 20, 15))
    # Eye
    pygame.draw.circle(screen, BLACK, (int(x + chick_width * 0.7), int(y + chick_height * 0.3)), 4)
    # Beak
    beak_points = [(x + chick_width - 2, y + 20), (x + chick_width + 10, y + 25), (x + chick_width - 2, y + 30)]
    pygame.draw.polygon(screen, ORANGE, beak_points)

def draw_egg(egg_rect):
    """Draws a single egg on the screen."""
    pygame.draw.ellipse(screen, WHITE, egg_rect)
    # Adding a thin black outline to the egg
    pygame.draw.ellipse(screen, BLACK, egg_rect, 2)

def display_score():
    """Renders and displays the current score."""
    score_text = score_font.render(f"Score: {score}", True, WHITE)
    screen.blit(score_text, (10, 10))

def display_game_over():
    """Displays the game over message."""
    over_text = game_over_font.render("GAME OVER", True, BLACK)
    text_rect = over_text.get_rect(center=(SCREEN_WIDTH / 2, SCREEN_HEIGHT / 2 - 40))
    screen.blit(over_text, text_rect)

    final_score_text = score_font.render(f"Final Score: {score}", True, BLACK)
    score_rect = final_score_text.get_rect(center=(SCREEN_WIDTH / 2, SCREEN_HEIGHT / 2 + 20))
    screen.blit(final_score_text, score_rect)

# --- Main Game Loop ---

# A custom event to trigger spawning new eggs
ADD_EGG_EVENT = pygame.USEREVENT + 1
pygame.time.set_timer(ADD_EGG_EVENT, 1200) # Spawn an egg every 1.2 seconds

while running:
    # --- 1. Event Handling ---
    for event in pygame.event.get():
        # Check if the user closed the window
        if event.type == pygame.QUIT:
            running = False

        # Handle jumping input only if the game is not over
        if not game_over:
            if event.type == pygame.KEYDOWN:
                # Jump if the up arrow is pressed and the chick is on the ground
                if event.key == pygame.K_UP and not is_jumping:
                    is_jumping = True
                    chick_y_velocity = jump_strength

            # Handle the custom event for adding new eggs
            if event.type == ADD_EGG_EVENT:
                # Create a new egg rectangle off the right side of the screen
                new_egg = pygame.Rect(SCREEN_WIDTH, ground_level - egg_height, egg_width, egg_height)
                # Store it as a dictionary to track its score status
                eggs.append({'rect': new_egg, 'passed': False})

    # --- 2. Game Logic ---
    if not game_over:
        # Chick physics (jumping and gravity)
        if is_jumping:
            chick_y_velocity += gravity
            chick_y += chick_y_velocity
            # Check if the chick has landed back on the ground
            if chick_y >= ground_level - chick_height:
                chick_y = ground_level - chick_height
                is_jumping = False
                chick_y_velocity = 0

        # Move eggs and update score
        for egg in eggs:
            # Move the egg to the left
            egg['rect'].x -= egg_speed
            # Check if the egg has passed the chick
            if egg['rect'].right < chick_x and not egg['passed']:
                score += 1
                egg['passed'] = True # Mark as passed to prevent scoring again

        # Remove eggs that have gone off-screen
        # We iterate over a copy [:] to safely remove items from the list
        for egg in eggs[:]:
            if egg['rect'].right < 0:
                eggs.remove(egg)

        # Collision detection
        chick_rect = pygame.Rect(chick_x, chick_y, chick_width, chick_height)
        for egg in eggs:
            if chick_rect.colliderect(egg['rect']):
                game_over = True

    # --- 3. Drawing / Rendering ---
    # Background
    screen.fill(SKY_BLUE)

    # Ground
    pygame.draw.rect(screen, GRASS_GREEN, (0, ground_level, SCREEN_WIDTH, 100))

    # Draw the elements
    draw_chick(chick_x, chick_y)
    for egg in eggs:
        draw_egg(egg['rect'])

    # Display score or game over screen
    if game_over:
        display_game_over()
    else:
        display_score()

    # --- 4. Update the Display ---
    # This draws everything from the buffer onto the screen
    pygame.display.flip()

    # --- 5. Control the Frame Rate ---
    # This ensures the game runs at a consistent speed
    clock.tick(FPS)

# --- 6. Quit the Game ---
pygame.quit()